In [1]:
#ML Libs & essential functions
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score
from sklearn.linear_model import LinearRegression


import numpy as np
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Input, Dense, Dropout

In [3]:
import zipfile

# unzip
with zipfile.ZipFile('healthy-diet-recipes-a-comprehensive-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('healthy_diet_recipes')

dishes_df = pd.read_csv('healthy_diet_recipes/All_Diets.csv')

In [5]:
#data display
separator = pd.DataFrame(['...'], columns=['Separator'])

display(dishes_df.head(5))
print('...')
display(dishes_df.tail(5))

,Diet_type,Recipe_name,Cuisine_type,Protein(g),Carbs(g),Fat(g),Extraction_day,Extraction_time
0,paleo,Bone Broth From 'Nom Nom Paleo',american,5.22,1.29,3.20,2022-10-16,17:20:09
1,paleo,"Paleo Effect Asian-Glazed Pork Sides, A Sweet ...",south east asian,181.55,28.62,146.14,2022-10-16,17:20:09
2,paleo,Paleo Pumpkin Pie,american,30.91,302.59,96.76,2022-10-16,17:20:09
3,paleo,Strawberry Guacamole recipes,mexican,9.62,75.78,59.89,2022-10-16,17:20:09
4,paleo,"Asian Cauliflower Fried ""Rice"" From 'Nom Nom P...",chinese,39.84,54.08,71.55,2022-10-16,17:20:09


...


,Diet_type,Recipe_name,Cuisine_type,Protein(g),Carbs(g),Fat(g),Extraction_day,Extraction_time
7801,dash,Brown Butter-Sunchoke Soup With Brussels Sprou...,italian,85.20,288.14,137.15,2022-10-16,20:40:44
7802,dash,Make-Your-Own-Salad,american,141.98,123.18,323.50,2022-10-16,20:40:44
7803,dash,Luis Buñuel Dry Martini,world,0.01,0.39,0.00,2022-10-16,20:40:44
7804,dash,Cornflake Semi-Fried Chicken Tenders,american,155.38,239.88,260.84,2022-10-16,20:40:44
7805,dash,Emeril's Classic Manhattan,american,0.02,0.83,0.00,2022-10-16,20:40:44


In [7]:
features = dishes_df[['Protein(g)', 'Carbs(g)', 'Fat(g)']].values

#preprocessing
scaler= MinMaxScaler()
scaled_feat= scaler.fit_transform(features)

#user data input
weight= float(input("Enter your weight in kilograms (kg): "))


#user input for predictions
# goal_protein = float(input("Enter your target protein quantity (g): "))
# goal_carbs = float(input("Enter your target carb quantity (g): "))
# goal_fat = float(input("Enter your target fat quantity (g): "))

#error check
if (goal_protein > (weight/3)):
    print("Error: your protein quantity selection could be unhealthy")

Enter your weight in kilograms (kg):  22


NameError: name 'goal_protein' is not defined

In [9]:
def create_sequences(data, target, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(target[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 3 
scaled_feat = np.array(scaled_feat)
X, y = create_sequences(scaled_feat, scaled_feat, seq_length)

# Reshape X for LSTM input
X = X.reshape((X.shape[0], X.shape[1], -1))

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(3))  # Output layer for protein, carbs, fat

model.compile(optimizer='adam', loss='mean_absolute_error')

# Ensure batch size is reasonable
model.fit(X, y, epochs=50, batch_size=32)  # Adjust batch size as needed

C:\Users\mmoni\Anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 0.0408
Epoch 2/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0383
Epoch 3/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0389
Epoch 4/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0388
Epoch 5/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0393
Epoch 6/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0397
Epoch 7/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0389
Epoch 8/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0379
Epoch 9/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0387
Epoch 10/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0386
Epoch 11/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0379
Epoch 12/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0389
Epoch 13/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0389
Epoch 14/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0384
Epoch 15/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - lo

In [13]:
from itertools import combinations

# Load the meals data from the CSV file
# Define the target protein value
target_protein = 100

# Find all possible combinations of meals
meal_combinations = []
for r in range(1, len(dishes_df) + 1):
    for combo in combinations(dishes_df.index, r):
        total_protein = dishes_df.loc[list(combo), 'Protein(g)'].sum()
        if total_protein == target_protein:
            meal_combinations.append(dishes_df.loc[list(combo), 'Recipe_name'].tolist())

# Print the combinations
print("Combinations of meals with protein values summing to 100:")
for combo in meal_combinations:
    print(combo)


KeyboardInterrupt

